In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
print("Siddharth Adhikari");

Siddharth Adhikari


In [8]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [10]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [11]:
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [12]:
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

In [13]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [14]:
X = pipeline.fit_transform(X)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [16]:
# I recommend to upgrade pip 
! pip install --upgrade pip 
# Installing tensorflow 
! pip install tensorflow 

In [17]:
import tensorflow as tf 
from tensorflow import keras 

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [19]:
classifier = Sequential()

In [20]:
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [21]:
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [22]:
classifier.add(Dense(1, activation = 'sigmoid'))

In [23]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [25]:
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 2s - loss: 0.5595 - accuracy: 0.7432 - val_loss: 0.4976 - val_accuracy: 0.7950 - 2s/epoch - 10ms/step
Epoch 2/200
225/225 - 1s - loss: 0.5033 - accuracy: 0.7921 - val_loss: 0.4758 - val_accuracy: 0.7950 - 969ms/epoch - 4ms/step
Epoch 3/200
225/225 - 1s - loss: 0.4842 - accuracy: 0.7935 - val_loss: 0.4609 - val_accuracy: 0.7950 - 571ms/epoch - 3ms/step
Epoch 4/200
225/225 - 0s - loss: 0.4699 - accuracy: 0.7972 - val_loss: 0.4479 - val_accuracy: 0.7950 - 494ms/epoch - 2ms/step
Epoch 5/200
225/225 - 0s - loss: 0.4607 - accuracy: 0.7939 - val_loss: 0.4353 - val_accuracy: 0.7962 - 460ms/epoch - 2ms/step
Epoch 6/200
225/225 - 0s - loss: 0.4501 - accuracy: 0.7979 - val_loss: 0.4260 - val_accuracy: 0.8050 - 446ms/epoch - 2ms/step
Epoch 7/200
225/225 - 0s - loss: 0.4418 - accuracy: 0.8054 - val_loss: 0.4164 - val_accuracy: 0.8087 - 453ms/epoch - 2ms/step
Epoch 8/200
225/225 - 0s - loss: 0.4339 - accuracy: 0.8082 - val_loss: 0.4075 - val_accuracy: 0.8125 - 447ms/epoch - 2ms

In [26]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 1s 2ms/step
[[0.25011477]
 [0.26174542]
 [0.14707749]
 [0.09585193]
 [0.11880509]]


In [27]:
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [28]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1522   73]
 [ 191  214]]


In [29]:
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.8 % of data was classified correctly
